In [1]:
isNotebook = '__file__' not in locals()

In [2]:
from systemtools.hayj import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.location import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser

In [3]:
logger = Logger()

In [4]:
TEST = True
initialDatasetVersion = 1 if TEST else 3
datasetVersion = "1.0"

In [5]:
dataRootPath = dataDir() + "/Twinews/" + "twinews" + str(initialDatasetVersion)

In [6]:
newsFiles = sortedGlob(dataRootPath + "/news/*.bz2")
bp(newsFiles)
bp(list(NDJson(random.choice(newsFiles)))[0].keys(), 5)
bp(list(NDJson(random.choice(newsFiles)))[0]['scrap'].keys(), 5)
if TEST:
    newsFiles = newsFiles[:10]

[
  /NoSave/Data/Twinews/twinews1/news/9news.com.bz2,
  /NoSave/Data/Twinews/twinews1/news/10news.com.bz2,
  ...,
  /NoSave/Data/Twinews/twinews1/news/zdnet.com.bz2,
  /NoSave/Data/Twinews/twinews1/news/zerohedge.com.bz2
]
dict_keys(['domain', 'lastUrlDomain', 'relevant', 'scrap', 'url', 'title', 'redirected', 'lastUrl'])
dict_keys(['meta_data', 'publish_date', 'meta_description', 'meta_lang', 'title', 'authors', 'meta_favicon', 'tags', 'text'])


In [7]:
usersFiles = sortedGlob(dataRootPath + "/users/*.bz2")
bp(usersFiles)
bp(list(NDJson(random.choice(usersFiles)))[0].keys(), 5)

[ /NoSave/Data/Twinews/twinews1/users/part0.bz2, /NoSave/Data/Twinews/twinews1/users/part1.bz2 ]
dict_keys(['tweet_count', 'location', 'bio', 'tweets', 'list_count', 'joindate_title', 'joindate_text', 'moment_count', 'notBotScore', 'verified', 'username', 'user_website', 'follower_count', 'following_count', 'favorite_count', 'user_id', 'datasetRelevanceScore', 'url', 'name', 'media_count', 'tweets_en_ratio', 'consecutive_old_tweets', 'avatar', 'news', 'user_twitter_website', 'date_limit'])


In [8]:
(hjuser, hjpass, hjhost) = getTipiMongoAuth()
(stuser, stpass, sthost) = getTipiStudentMongoAuth()

In [9]:
mongoCollectionKwargs = \
{
    "giveTimestamp": False,
    "giveHostname": False,
    "verbose": True,
    "logger": logger,
    "user": hjuser,
    "password": hjpass,
    "host": hjhost,
}

In [10]:
news = MongoCollection("twinews", "news", version=datasetVersion,
                        indexOn=["url"],
                       indexNotUniqueOn=["domain", "lastUrlDomain", "lastUrl", "isGoodArticle"],
                        **mongoCollectionKwargs)
users = MongoCollection("twinews", "users", version=datasetVersion, indexOn=["user_id"],
                        indexNotUniqueOn=["datasetRelevanceScore", "notBotScore"],
                        **mongoCollectionKwargs)

twinews news (version 1.0) initialised.
twinews users (version 1.0) initialised.


In [11]:
news.resetCollection(security=False)
users.resetCollection(security=False)

In [12]:
assert len(news) == 0
assert len(users) == 0

In [13]:
def preprocessNews(row, logger=None, verbose=True):
    try:
        if isinstance(row, str):
            text = row
        else:
            if dictContains(row, "scrap"):
                row = row['scrap']
            if dictContains(row, "text"):
                text = row['text']
            else:
                raise Exception("No text found in " + b(row))
        text = newsPreclean(text)
        isGood = isGoodArticle(text)
        rawText = text
        (text, tokens) = newsParser.parseNews(rawText, logger=logger, verbose=verbose)
        return (rawText, text, tokens, isGood)
    except Exception as e:
        logException(e, logger, verbose=verbose)

In [14]:
for file in pb(shuffle(newsFiles), logger=logger):
    for row in NDJson(file):
        current = dictSelect(row, {'domain', 'lastUrlDomain', 'url', 'title', 'redirected', 'lastUrl'})
        scrap = row['scrap']
        current["title"] = preprocess(current["title"], doRemoveUrls=True, unescapeHtml=True,
                              removeHtml=True, doQuoteNormalization=True,
                              doReduceBlank=True, keepNewLines=False, logger=logger)
        (rawText, text, tokens, isGood) = preprocessNews(scrap, logger=logger)
        current["rawText"] = rawText
        current["text"] = text
        current["tokens"] = tokens
        current["isGoodArticle"] = isGood
        news.insert(current)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loading the GoogArticle model wich is a <class 'sklearn.svm._classes.LinearSVC'>


 10% [==                  ] (45.899s left)
 20% [====                ] (21.32s left)
 30% [======              ] (13.16s left)
 40% [========            ] (8.595s left)
 50% [==========          ] (6.32s left)
 60% [============        ] (4.813s left)
 70% [==============      ] (6.351s left)
 80% [================    ] (3.709s left)
 90% [==================  ] (1.739s left)
100% [====================] (total duration: 18.7s, mean duration: 1.869s)
